In [178]:
import pandas as pd
import numpy as np
import numpy_financial as npf
from sklearn.linear_model import LinearRegression, Ridge,LogisticRegression
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures,LabelEncoder
from imblearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImPipeline

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV,cross_validate
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn.metrics import recall_score,accuracy_score, precision_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
from sklearn.metrics import log_loss
from sklearn.metrics import RocCurveDisplay
import warnings

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [179]:
df = pd.read_csv('Data/Loan_Default.csv', index_col=0)

In [180]:
df.head()

,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,loan_amount,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
ID,,,,,,,,,,,,,,,,,,,,,
24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,116500,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,206500,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,406500,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,456500,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,696500,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


In [181]:
columns = list(df.columns)
columns

['year',
 'loan_limit',
 'Gender',
 'approv_in_adv',
 'loan_type',
 'loan_purpose',
 'Credit_Worthiness',
 'open_credit',
 'business_or_commercial',
 'loan_amount',
 'rate_of_interest',
 'Interest_rate_spread',
 'Upfront_charges',
 'term',
 'Neg_ammortization',
 'interest_only',
 'lump_sum_payment',
 'property_value',
 'construction_type',
 'occupancy_type',
 'Secured_by',
 'total_units',
 'income',
 'credit_type',
 'Credit_Score',
 'co-applicant_credit_type',
 'age',
 'submission_of_application',
 'LTV',
 'Region',
 'Security_Type',
 'Status',
 'dtir1']

In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148670 entries, 24890 to 173559
Data columns (total 33 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   year                       148670 non-null  int64  
 1   loan_limit                 145326 non-null  object 
 2   Gender                     148670 non-null  object 
 3   approv_in_adv              147762 non-null  object 
 4   loan_type                  148670 non-null  object 
 5   loan_purpose               148536 non-null  object 
 6   Credit_Worthiness          148670 non-null  object 
 7   open_credit                148670 non-null  object 
 8   business_or_commercial     148670 non-null  object 
 9   loan_amount                148670 non-null  int64  
 10  rate_of_interest           112231 non-null  float64
 11  Interest_rate_spread       112031 non-null  float64
 12  Upfront_charges            109028 non-null  float64
 13  term                     

In [183]:
df['Upfront_charges%']=df['Upfront_charges']/df['loan_amount']

In [184]:
df.isna().sum()

year                             0
loan_limit                    3344
Gender                           0
approv_in_adv                  908
loan_type                        0
loan_purpose                   134
Credit_Worthiness                0
open_credit                      0
business_or_commercial           0
loan_amount                      0
rate_of_interest             36439
Interest_rate_spread         36639
Upfront_charges              39642
term                            41
Neg_ammortization              121
interest_only                    0
lump_sum_payment                 0
property_value               15098
construction_type                0
occupancy_type                   0
Secured_by                       0
total_units                      0
income                        9150
credit_type                      0
Credit_Score                     0
co-applicant_credit_type         0
age                            200
submission_of_application      200
LTV                 

### Impute missing values for rate of interest,LTV,Upfront Charges%, interest rate spread and loan limit category using pipeline

In [185]:
nummedian = ['rate_of_interest','LTV','Upfront_charges%','dtir1']
nummean =['Interest_rate_spread']

In [186]:
catcols = ['loan_limit']

In [187]:
X = df.drop('Status', axis =1)
y = df['Status']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state =42,test_size = 0.25)

In [188]:
subpipe_mean =Pipeline(steps =[
    ('Impute',SimpleImputer(strategy='mean'))      
])

subpipe_median =Pipeline(steps =[    
    ('Impute',SimpleImputer(strategy='median'))
])
    
subpipe_cat =Pipeline([
    ('Impute',SimpleImputer(strategy='most_frequent'))
])

In [189]:
CT = ColumnTransformer(transformers=[
    ('subpipe_mean', subpipe_mean,nummean),
    ('subpipe_median', subpipe_median,nummedian),
    ('subpipe_cat', subpipe_cat,catcols)
],remainder='passthrough',verbose_feature_names_out=False)

### After imputing missing values convert X_train data to a dataframe for other calculations

In [190]:
CT.set_output(transform='pandas')
X_train_trans = CT.fit_transform(X_train)
X_train_trans

,Interest_rate_spread,rate_of_interest,LTV,Upfront_charges%,dtir1,loan_limit,year,Gender,approv_in_adv,loan_type,...,Secured_by,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,Region,Security_Type
ID,,,,,,,,,,,,,,,,,,,,,
148781,0.553500,4.375,57.750397,0.002307,30.0,cf,2019,Joint,nopre,type1,...,home,1U,10560.0,CRIF,625,EXP,45-54,not_inst,North,direct
40828,0.775800,4.375,78.846154,0.021032,44.0,cf,2019,Female,pre,type1,...,home,2U,3240.0,CRIF,567,CIB,25-34,to_inst,North,direct
133157,0.704600,3.625,73.437500,0.000000,39.0,cf,2019,Joint,nopre,type2,...,home,1U,NaN,EXP,692,EXP,55-64,not_inst,North,direct
113792,0.550700,3.625,64.737991,0.010367,36.0,cf,2019,Joint,nopre,type1,...,home,1U,5820.0,CIB,893,EXP,55-64,to_inst,North,direct
86062,0.152500,3.625,58.359873,0.023379,38.0,cf,2019,Joint,nopre,type1,...,home,1U,5160.0,CRIF,719,EXP,>74,to_inst,south,direct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144769,0.441735,3.990,69.345238,0.009932,38.0,cf,2019,Male,nopre,type1,...,home,1U,1560.0,CRIF,552,CIB,65-74,to_inst,North,direct
128584,1.339500,4.500,87.397541,0.000000,52.0,cf,2019,Joint,nopre,type2,...,home,1U,7560.0,CRIF,545,EXP,45-54,to_inst,south,direct
156822,1.328700,4.990,79.362416,0.021353,48.0,cf,2019,Sex Not Available,nopre,type1,...,home,1U,4800.0,EXP,675,EXP,55-64,to_inst,south,direct


### After imputing missing values convert X_test data to a dataframe for other calculations

In [118]:
CT.set_output(transform='pandas')
X_test_trans = CT.fit_transform(X_test)
X_test_trans

,Interest_rate_spread,rate_of_interest,LTV,Upfront_charges%,dtir1,loan_limit,year,Gender,approv_in_adv,loan_type,...,Secured_by,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,Region,Security_Type
ID,,,,,,,,,,,,,,,,,,,,,
49802,1.447400,4.990,84.420290,0.021143,41.0,cf,2019,Female,nopre,type2,...,home,1U,1740.0,EXP,679,CIB,45-54,to_inst,North,direct
171958,0.441700,3.990,88.246269,0.010645,44.0,cf,2019,Sex Not Available,pre,type1,...,home,1U,5880.0,EXP,853,EXP,25-34,to_inst,south,direct
148174,0.452200,3.990,19.279661,0.028571,41.0,cf,2019,Sex Not Available,nopre,type1,...,home,1U,2340.0,CRIF,598,EXP,65-74,to_inst,south,direct
78500,0.622000,4.250,36.228814,0.013402,32.0,cf,2019,Female,nopre,type1,...,home,1U,5640.0,EXP,776,EXP,65-74,to_inst,North,direct
64562,-0.224100,3.375,81.921241,0.000000,48.0,cf,2019,Female,nopre,type1,...,home,1U,7140.0,EXP,653,CIB,35-44,not_inst,North,direct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95608,0.522700,4.125,79.233871,0.028443,7.0,cf,2019,Male,nopre,type1,...,home,1U,8100.0,CRIF,656,CIB,55-64,to_inst,North,direct
72315,0.439500,4.875,74.100257,0.000000,46.0,cf,2019,Joint,nopre,type1,...,home,1U,10680.0,EXP,578,EXP,55-64,not_inst,south,direct
157267,0.441418,3.990,50.932836,0.009923,39.0,cf,2019,Sex Not Available,nopre,type2,...,home,1U,NaN,EXP,583,EXP,55-64,to_inst,south,direct


In [145]:
df_train = pd.concat([X_train_trans,y_train],axis=1)
df_test = pd.concat([X_test_trans,y_test],axis=1)

### Estimate upfront charges using upfront charges % for X_train and X_test

In [146]:
df_train['Upfront_charges'].fillna(df_train['loan_amount'] * df_train['Upfront_charges%'], inplace=True)

In [147]:
df_test['Upfront_charges'].fillna(df_test['loan_amount'] * df_test['Upfront_charges%'], inplace=True)

### Estimate property value using LTV for X_train and X_test

In [148]:
df_train['property_value'].fillna(df_train['loan_amount']*100 / df_train['LTV'], inplace=True)

In [149]:
df_test['property_value'].fillna(df_test['loan_amount']*100 / df_test['LTV'], inplace=True)

### Missing Values of Monthly Income for X_train and X_test

As missing values of monthly income varies from person to person cannot be determine directly. Also there are some records have 0 income which is impossible. Therefore those rows removed from the dataframe.

In [151]:
df_train = df_train[df_train['income']>0]

In [152]:
df_test = df_test[df_test['income']>0]

### Missing Values of approve in advance, loan purpose, term and neg_ammortization for X_train and X_test

As those are much personalized information and considering the number of missing values even less than 1%, those records have removed from the database

In [153]:
df_train = df_train.dropna(subset=['approv_in_adv','loan_purpose','term','Neg_ammortization'])

In [154]:
df_test = df_test.dropna(subset=['approv_in_adv','loan_purpose','term','Neg_ammortization'])

In [155]:
df_train.shape

(102501, 36)

In [156]:
X_train_trans.isna().sum()

Interest_rate_spread         0
rate_of_interest             0
LTV                          0
Upfront_charges%             0
dtir1                        0
loan_limit                   0
year                         0
Gender                       0
approv_in_adv                0
loan_type                    0
loan_purpose                 0
Credit_Worthiness            0
open_credit                  0
business_or_commercial       0
loan_amount                  0
Upfront_charges              0
term                         0
Neg_ammortization            0
interest_only                0
lump_sum_payment             0
property_value               0
construction_type            0
occupancy_type               0
Secured_by                   0
total_units                  0
income                       0
credit_type                  0
Credit_Score                 0
co-applicant_credit_type     0
age                          0
submission_of_application    0
Region                       0
Security

### Calculate Monthly Loan Payment for X_train and X_test

Calculate monthly premium based on rate of interest, term and loan amount using loan ammmortization

In [157]:
df_train['monthly_premium'] = npf.pmt(df_train['rate_of_interest']/(12*100), df_train['term'], df_train['loan_amount'])*-1

In [158]:
df_test['monthly_premium'] = npf.pmt(df_test['rate_of_interest']/(12*100), df_test['term'], df_test['loan_amount'])*-1

Calculate monthly premium to income ratio

In [159]:
df_train['PTI']=df_train['monthly_premium']/df_train['income']

In [160]:
df_test['PTI']=df_test['monthly_premium']/df_test['income']

Cleansing PTI column, where PMI column greater than 1 is not possible. If greater than 1 means that loan premium is greater than the income. These impossible scenarios to remove from the dataframe.

In [161]:
df_train = df_train[df_train['PTI']<1]

In [162]:
df_test = df_test[df_test['PTI']<1]

In [163]:
df_train.shape

(102501, 36)

In [164]:
df_test.shape

(34172, 36)

### Composition of Target Column

In [168]:
df_train['Status'].value_counts()

0    77315
1    25186
Name: Status, dtype: int64

In [169]:
df_test['Status'].value_counts()

0    25888
1     8284
Name: Status, dtype: int64

In [170]:
df_train.to_csv('data/Loan_Default_train_cleaned.csv')

In [171]:
df_test.to_csv('data/Loan_Default_test_cleaned.csv')